## 1. Supply Chain mapping with neo4j

In [206]:
from xbbg import blp
import neo4j
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import nxneo4j as nnx
import networkx as nx
from typing import Union, Tuple, List, Optional, Set
from typing import List, Tuple, Iterable, Dict
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
import os
from pathlib import Path


uri = r"bolt://localhost:7687/"
user = "neo4j"
LOGPATH = Path("./log/")
if not LOGPATH.exists():
    LOGPATH.mkdir()
LOGFILE = LOGPATH.joinpath("log.txt")
logging.basicConfig(filename=LOGFILE.as_posix())

METADATA_FLDS = ['name', 'short_name', 'security_name', 'long_comp_name',
    'name_chinese_traditional', 'name_kanji', 'industry_sector',
    'industry_group', 'industry_subgroup', 
    'name_chinese_simplified', 'ud_em_sector', 'ud_research_note_date',
    'ud_msci_market_classification', 'ud_water_sector', 'ud_esg_score',
    'pct_independent_directors', 'ud_esg_note_date',
    'gics_sector_name', 'gics_industry_group_name', 'gics_iindustry_name', 
    'gics_sub_industry_name', 'cur_adj_mkt_cap', 
    'best_cur_ev_to_ebitda', 'best_pe_ratio', 'est_pe_nxt_yr', 
    'est_pe_cur_yr', 'five_yr_avg_price_earnings', 'best_px_sales_ratio',
    'fcf_yield_with_cur_entp_val', 'px_to_book_ratio',
    'crncy_adj_mkt_cap'
    ]

DEFAULT_NNX_CONFIG = {"node_label": "Company",
    "relationship_type": "supply",
    }

class Neo4jQueries:
    def create_database(driver, database_name: str='neo4j') -> None:
        with driver.session(default_access_mode=neo4j.READ_ACCESS) as session:
            session.run(f"CREATE {database_name} IF NOT EXISTS")
        
    def __init__(self, driver: GraphDatabase.driver, database_name: str=None):
        self.driver = driver
        if database_name:
            self.create_graph(database_name=database_name)

    def add_node():
        raise NotImplementedError

    

class SupplyChainRelation:

    @staticmethod
    def _metadata(tickers: Iterable[str], max_len: int=100, max_workers: int=16) -> pd.DataFrame:
        """get the metadata of the tickers
        :param tickers: iterable of the tickers
        :param max_size: max no of tickers to be sent to the API at once
        :param max_workers: max workers for multi threaded calculation
        """
        if len(tickers) <= max_len:
            res = blp.bdp(tickers, flds=METADATA_FLDS)
        else:
            chunks = [tickers[i: i+max_len] for i in range(0, len(tickers), max_len)]
            if max_workers:
                with ThreadPoolExecutor(max_workers=max_workers) as executor:
                    futures = [executor.submit(blp.bdp, chunk, flds=METADATA_FLDS) for chunk in chunks]
                    res = pd.concat([future.result() for future in as_completed(futures)])
            else:
                res = pd.concat([blp.bdp(chunk, flds=METADATA_FLDS) for chunk in chunks])
        return res

    def __init__(self, 
        tickerlist: Optional[Iterable[str]]=None, 
        max_workers: int=16,
        neo4j_config: Optional[Dict[str, str]]=None):
        self.tickerlist = tickerlist
        self.__all__ = ['get_cutomers', 'get_suppliers'] # TODO - update this when done
        self.max_workers = max_workers
        self._driver = None
        if neo4j_config:
            self._driver = GraphDatabase.driver(uri=neo4j_config.get('uri'),
                auth=(neo4j_config.get('username', 'neo4j'), 
                    neo4j_config.get('password')))

    @property
    def driver(self):
        return self._driver

    @driver.setter
    def driver(self, new_driver: GraphDatabase):
        self._driver = new_driver

    @staticmethod
    def _customer_list(ticker: str, 
        top_k: int=50,
        ignore_error: bool=False) -> pd.DataFrame:
        """returns dataframe of the customers
        :param ticker: takes the BBG ID or BBG ticker
        :param top_k: top k relations to be returned
        """
        try:
            df = blp.bds(ticker,"supply_chain_customers",
                        supply_chain_sum_count_override=top_k)
            df.columns = ['partner_ticker']
        except Exception as e:
            if ignore_error:
                logging.error(f"The following happened while calling _rel_amount({ticker}, {top_k}):\n{e}\n")
                pass
            else:
                raise e
        return df
    
    @staticmethod
    def _supplier_list(ticker: str, 
        top_k: int=50,
        ignore_error: bool=False) -> pd.DataFrame:
        """returns dataframe of the suppliers
        :param ticker: takes the BBG ID or BBG ticker
        :param top_k: top k relations to be returned
        """
        try:
            df = blp.bds(ticker, "supply_chain_suppliers",
                        supply_chain_sum_count_override=top_k)
            df.columns = ['partner_ticker']
        except Exception as e:
            if ignore_error:
                logging.error(f"The following happened while calling _rel_amount({ticker}, {top_k}):\n{e}\n")
                pass
            else:
                raise e
        return df
    
    @staticmethod
    def _rel_amount(head: str, tail: str, relationship: str='C',
        ignore_error: bool=True):
        """
        :param head: head of the relation
        :param tail: tail of the relation
        :param relationship: takes 'C' (customer) or 'S' (supplier). 
        
        Note:
            Order of head and tail matters. (head, tail, relationship) implies
            that tail has_relationship with head. e.g. A, B, 'C' means B is 
            customer of A. Reversing A and B will result in returning None
        """
        try:
            tmp = blp.bdp(head, "RELATIONSHIP_AMOUNT",
                        relationship_override=relationship,
                        quantified_override='Y',
                        eqy_fund_crncy='USD',
                        related_company_override=tail)
            tmp.index = [tail]
            return tmp
        except Exception as e:
            if ignore_error:
                logging.error(f"The following happened while calling _rel_amount({head}, {tail}, {relationship}):\n{e}\n")
                pass
            else:
                raise e

    @staticmethod
    def _rel_perc(head: str, tail: str, 
        relationship: str='C',
        ignore_error: bool=False):
        """
        :param head: head of the relation
        :param tail: tail of the relation
        :param relationship: takes 'C' (customer) or 'S' (supplier). 
        
        Note:
            Order of head and tail matters. (head, tail, relationship) implies
            that tail has_relationship with head. e.g. A, B, 'C' means B is 
            customer of A. Reversing A and B will result in returning None
        """
        try:
            tmp = blp.bdp(head, "SUPPLY_CHAIN_REVENUE_PERCENTAGE",
                        relationship_override=relationship,
                        quantified_override='Y',
                        eqy_fund_crncy='USD',
                        related_company_override=tail)
            tmp.index = [tail]
            return tmp
        except Exception as e:
            if ignore_error:
                logging.error(f"The following happened while calling _rel_amount({head}, {tail}, {relationship}):\n{e}\n")
                pass
            else:
                raise e
        
    def __contribution(self,
        ticker: str, 
        top_k: int=50, 
        relationship: str='C',
        return_type: str='amount') -> pd.DataFrame:
        """get contribution by customer
        :param ticker: ticker to get information about
        :param top_k: max number of relations to be returned
        :param relationship: takes 'C' - customer or 'S' - supplier
        :param return_type: str, takes 'amount' and 'percent'
        """
        tmps = []
        if relationship in ['C', 'customers']:
            partners = self._customer_list(ticker, top_k=top_k).partner_ticker
            if return_type in ['amount']:
                with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                    futures = [executor.submit(self._rel_amount, ticker, p, relationship) 
                        for p in partners]
            elif return_type in ['percent']:
                with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                    futures = [executor.submit(self._rel_perc, ticker, p, relationship) 
                        for p in partners]
            else: raise NotImplementedError
            
        elif relationship in ['S', 'suppliers']:
            partners = self._supplier_list(ticker, top_k=top_k).partner_ticker
            if return_type in ['amount']:
                with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                    futures = [executor.submit(self._rel_amount, ticker, p, relationship) 
                        for p in partners]
            elif return_type in ['percent']:
                with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                    futures = [executor.submit(self._rel_perc, ticker, p, relationship) 
                        for p in partners]
            else: raise NotImplementedError
        else: raise NotImplementedError
        tmps = [future.result() for future in as_completed(futures)]
        df = pd.concat(tmps)
        return df

    def _customer_contribution(self, 
        ticker: str, 
        top_k: int=50, 
        return_type: str='amount') -> pd.DataFrame:
        """get contribution by customer
        :param ticker: ticker to get information about
        :param top_k: max number of relations to be returned
        :param return_type: str, takes 'amount' and 'percent'
        """
        return self.__contribution(ticker=ticker, top_k=top_k, 
            relationship='C', 
            return_type=return_type)

    @classmethod
    def get_customer_contribution(cls, ticker: str, 
        top_k: int=50, 
        return_type='amount', **kwargs):
        """classmethod version of _customer_contribution"""
        return cls(**kwargs)._customer_contribution(
            ticker, top_k=top_k, return_type=return_type)

    def _supplier_contribution(self, 
        ticker: str, 
        top_k: int=50, 
        return_type: str='amount') -> pd.DataFrame:
        """get contribution by customer
        :param ticker: ticker to get information about
        :param top_k: max number of relations to be returned
        :param return_type: str, takes 'amount' and 'percent'
        """
        return self.__contribution(ticker=ticker, top_k=top_k, 
            relationship='S', 
            return_type=return_type)

    @classmethod
    def get_supplier_contribution(cls, ticker: str, 
        top_k: int=50, 
        return_type='amount', **kwargs):
        """classmethod version of _supplier_contribution"""
        return cls(**kwargs)._supplier_contribution(
            ticker, top_k=top_k, return_type=return_type)

    def __add_ticker_partners(self, ticker: str, 
        edges: List[Dict], 
        top_k: int=50,
        nodes: Set=set(),
        verbose: bool=False,
        ignore_error: bool=True):
        """add relations to graph and return new nodes
        :param ticker: ticker to be processed
        :param G: graph object
        :param top_k: top_k relations to be returned
        :param nodes: unique nodes in the graph
        :param verbose: print which ticker is being processed
        """
        nodes.add(ticker)
        if verbose: print(ticker)
        suppliers_amount = self._supplier_contribution(ticker, top_k=top_k).reset_index()
        suppliers_percent = self._supplier_contribution(ticker, top_k=top_k, return_type='percent').reset_index()
        suppliers = suppliers_amount.merge(suppliers_percent, on=['index']).set_index('index')
        for supplier, row in suppliers.iterrows():
            try:
                if verbose: print(ticker, supplier)
                nodes.add(supplier)
                edges.append(dict(node1=supplier, node2=ticker, 
                    type='supply', label='supply',
                    source='Bloomberg',
                    percent_of_supplier=row.supply_chain_revenue_percentage,
                    amount=row.relationship_amount))
            except Exception as e:
                if ignore_error:
                    logging.error(e)
                    pass
                else:
                    raise e
        customers_amount = self._customer_contribution(ticker, top_k=top_k).reset_index()
        customers_percent = self._customer_contribution(ticker, top_k=top_k, return_type='percent').reset_index()
        customers = customers_amount.merge(customers_percent, on=['index']).set_index('index')
        for customer, row in customers.iterrows():
            if verbose: print(ticker, customer)
            try:
                nodes.add(customer)
                edges.append(dict(node1=ticker, node2=customer, 
                    type='supply', label='supply',
                    source='Bloomberg',
                    percent_of_supplier=row.supply_chain_revenue_percentage,
                    amount=row.relationship_amount))
            except Exception as e:
                if ignore_error:
                    logging.error(e)
                    pass
                else:
                    raise e
        
        return edges, nodes

    def get_graph(self, top_k=50, 
        max_workers: int=16,
        with_metadata: bool=True,
        verbose: bool=True,
        to_neo4j: bool=False,
        ignore_error: bool=True,
        nxneo4j_config: Dict=DEFAULT_NNX_CONFIG):
        """returns a graph object of the supply chain relationship
        :param useAmount: is True, will use the amount of the relationship as 
            weight of edges
        :param top_k: Max # of partners retrieves. 50 by default
        """
        
        if to_neo4j:
            G = nnx.DiGraph(self.driver, config=nxneo4j_config) 
        else:
            G = nx.DiGraph()
        nodes, edges = set(), list()
        
        if max_workers:
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                futures = [executor.submit(self.__add_ticker_partners, ticker, edges, top_k, nodes, verbose, ignore_error)
                    for ticker in self.tickerlist]
                for future in as_completed(futures):
                    try:
                        new_edges, new_nodes = future.result()
                        nodes.add(new_nodes)
                        edges.append(new_edges)
                    except Exception as e:
                        if ignore_error: 
                            logging.error(f"""The following error happened while calling get_graph(top_k={top_k}, with_metadata={with_metadata}, verbose={verbose}, to_neo4j={to_neo4j}, ignore_error={ignore_error}))
                            {e}
                            """)
            
                        else: raise e
        else:
            for ticker in self.tickerlist:
                try:
                    edges, nodes = self.__add_ticker_partners(ticker, edges, top_k, nodes, verbose, ignore_error)
                except Exception as e:
                    if ignore_error: 
                        logging.error(f"""The following error happened while calling get_graph(top_k={top_k}, with_metadata={with_metadata}, verbose={verbose}, to_neo4j={to_neo4j}, ignore_error={ignore_error}))
                        {e}
                        """)
            
                    else: raise e
        metadata = self._metadata(list(nodes))
        for node, row in metadata.iterrows():
            G.add_node(node, node_label='Company', labels=['Company', row.industry_sector, row.industry_group, row.industry_subgroup], 
                data_source='Bloomberg', **row.to_dict())
        for edge in edges:
            G.add_edge(data_source='Bloomberg', relationship_type='supply', 
                name='supply', **edge)
        return G, metadata

        

In [183]:
tickers = np.unique([f"{t} Equity" for t in 
    blp.bds("SOX Index", "indx_members").member_ticker_and_exchange_code] + \
        [f"{t} Equity" for t in 
    blp.bds("S5INFT Index", "indx_members").member_ticker_and_exchange_code])

In [208]:
sc = SupplyChainRelation(tickers,
    neo4j_config={'uri': r"bolt://localhost:7687", 
        "password": input("password"),
        "username": "neo4j"})
G, metadata = sc.get_graph(verbose=True, max_workers=0, to_neo4j=True)

AAPL UW Equity
AAPL UW Equity 2382 TT Equity
AAPL UW Equity 2330 TT Equity
AAPL UW Equity 005930 KS Equity
AAPL UW Equity 4958 TT Equity
AAPL UW Equity SWKS US Equity
AAPL UW Equity 3711 TT Equity
AAPL UW Equity 2317 TT Equity
AAPL UW Equity QCOM US Equity
AAPL UW Equity 034220 KS Equity
AAPL UW Equity 011070 KS Equity
AAPL UW Equity 002475 CH Equity
AAPL UW Equity 4938 TT Equity
AAPL UW Equity STM FP Equity
AAPL UW Equity 000660 KS Equity
AAPL UW Equity 300433 CH Equity
AAPL UW Equity AVGO US Equity
AAPL UW Equity 002241 CH Equity
AAPL UW Equity 6753 JP Equity
AAPL UW Equity 2324 TT Equity
AAPL UW Equity JBL US Equity
AAPL UW Equity BT/A LN Equity
AAPL UW Equity T CN Equity
AAPL UW Equity 9984 JP Equity
AAPL UW Equity 9433 JP Equity
AAPL UW Equity TEF SM Equity
AAPL UW Equity T US Equity
AAPL UW Equity VOD LN Equity
AAPL UW Equity ORA FP Equity
AAPL UW Equity CEC GR Equity
AAPL UW Equity RCI/B CN Equity
AAPL UW Equity 762 HK Equity
AAPL UW Equity TMUS US Equity
AAPL UW Equity EN FP Eq

In [204]:
len(list(G.edges()))

0

## 2. Keyword mapping with neo4j

In [3]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(r"C:\Users\p.peng\FinancialModelingPrep\data\tickers.db")
df = pd.read_sql("SELECT * FROM `all_company_profiles`", conn)

In [4]:
from keybert import KeyBERT
from typing import Tuple, Union, Optional, Set, Dict
model = KeyBERT()

STOPWORDS = ['company', 'holdings', 'limited', 'ltd',
    'group']

def extract_keywords(doc: str, 
    ngram_range: Tuple[int, int]=(1, 1),
    stop_words: Optional[Set]=STOPWORDS,
    **kwargs):
    return model.extract_keywords(doc, 
        keyphrase_ngram_range=ngram_range, 
        stop_words=stop_words, **kwargs)

In [10]:
help(model.extract_keywords)

Help on method extract_keywords in module keybert._model:

extract_keywords(docs: Union[str, List[str]], candidates: List[str] = None, keyphrase_ngram_range: Tuple[int, int] = (1, 1), stop_words: Union[str, List[str]] = 'english', top_n: int = 5, min_df: int = 1, use_maxsum: bool = False, use_mmr: bool = False, diversity: float = 0.5, nr_candidates: int = 20, vectorizer: sklearn.feature_extraction.text.CountVectorizer = None, highlight: bool = False, seed_keywords: List[str] = None) -> Union[List[Tuple[str, float]], List[List[Tuple[str, float]]]] method of keybert._model.KeyBERT instance
    Extract keywords and/or keyphrases
    
    I would advise you to iterate over single documents as they
    will need the least amount of memory. Even though this is slower,
    you are not likely to run into memory errors.
    
    There is an option to extract keywords for multiple documents
    that is faster than extraction for multiple single documents.
    However, this method assumes that yo

In [10]:
add_node('test', {'foundedIn':2014, 'activelyTrade': False})

In [ ]:
"""OPTIONAL MATCH (n:Company {name:"test"})
RETURN n IS NOT NULL AS Predicate"""

In [32]:
driver = GraphDatabase.driver(uri=r"bolt://localhost:7687", 
        auth=("neo4j", input("password")))



res = node_exists(["Company"], {"name": "test1"})

In [48]:
print(add_relation(source={'label': 'Company', 'node_id': 'test_source'}, 
target={'label': 'Topic', 'node_id': 'test_target'}, 
relation_props={'type': 'mention'},
relation_type='related',
directed=True
    ))

MATCH 
        (a:Company),
        (b:Topic)
    WHERE a.node_id = test_source AND b.node_id = test_target
    CREATE (a)-[r:related{{'type': 'mention'}}] ->(b)
    


In [100]:
from neo4j import GraphDatabase, Query
import nxneo4j as nnx
import datetime as dt
from typing import List, Dict, Union, Tuple, Iterable
import numpy as np

driver = GraphDatabase.driver(uri=r"bolt://localhost:7687", 
        auth=("neo4j", input("password")))
DEFAULT_NNX_CONFIG = {"node_label": "Company",
    "relationship_type": "related_to",
    }
G = nnx.Graph(driver, config=DEFAULT_NNX_CONFIG)


def node_exists(node_labels: Iterable[str], prop: Dict={}, 
    driver: GraphDatabase.driver=driver, verbose: bool=False):
    """check if node exists"""
    props = ", ".join([f"""{k}: \"{v}\"""" for k, v in prop.items() if type(v) in [str]] +\
            [f"{k}: {v}" for k, v in prop.items() 
            if type(v) in [float, int]] +\
            [f"{k}: {int(v)}" for k, v in prop.items() 
            if type(v) in [bool]])
    query = f"""OPTIONAL MATCH (n:{":".join(node_labels)}\u007b{props}\u007d)
    RETURN n IS NOT NULL AS Predicate"""
    if verbose: print(query)
    with driver.session() as session:
        result = session.run(Query(query)).single().value()
    return result

def add_node(node_id, metadata: Dict, 
    node_labels: Iterable[int]=['Company'], 
    match_field: Iterable[str]=['node_id'],
    driver=driver,
    verbose: bool=False):
    """add node to the graph if doesn't exist"""
    labels = ": ".join(node_labels)
    if not node_exists(node_labels=node_labels, 
    prop={k:v for k, v in metadata.items() if k in match_field},
    verbose=verbose):
        with driver.session() as session:
            props= ", ".join([f"""{k}: \"{v.replace("\"", "'")}\"""" 
            for k, v in metadata.items() if type(v) in [str]] +\
                [f"""node_id: \"{node_id}\""""] +\
                [f"{k}: {v}" for k, v in metadata.items() 
                if type(v) in [float, int]] +\
                [f"{k}: {int(v)}" for k, v in metadata.items() 
                if type(v) in [bool]])
            query = f"""CREATE (n:{labels} \u007b{props}\u007d)"""
            if verbose: print(query)
            session.run(Query(query))
        if verbose: print(f"added {node_id}")
    if verbose: print(f"failed to add {node_id}")

def add_relation(source: Dict, target: Dict, 
    relation_props: Dict,
    relation_type: str, 
    directed: bool=True, 
    driver: GraphDatabase.driver=driver,
    verbose: bool=False) -> None:
    """add relation to graph
    :param source: dictionary of the properties for the source, 
        must contain {'label': $label}
    :param target: dictionary of the properties for the target
    :paramm relation_type: type of the relation
    :param directed: boolean
    :param driver: GraphDatabase.driver object
    """
    a_cond = "AND ".join([f"a.{k} = '{v}'" 
        for k, v in source.items() if k != 'label']) # TODO - add
    b_cond = "AND ".join([f"b.{k} = '{v}'" 
        for k, v in target.items() if k != 'label'])
    arr = '->' if directed else '-'
    rel_props = ", ".join([f"""{k}: \"{v.replace("\"", "'")}\"""" for k, v in relation_props.items() if type(v) in [str]] +\
                [f"{k}: {v if v else ''}" for k, v in relation_props.items() 
                if type(v) in [float, int] ] +\
                [f"{k}: {int(v)}" for k, v in relation_props.items() 
                if type(v) in [bool]])
    query = f"""MATCH 
        (a:{source.get("label")}),
        (b:{target.get("label")})
    WHERE {a_cond} AND {b_cond}
    CREATE (a)-[r:{relation_type}\u007b{rel_props}\u007d]{arr}(b)
    """
    if verbose: print(query)
    with driver.session() as session:
        session.run(Query(query))

def add_ticker_with_keyword(ticker, 
    metadata: Dict, 
    max_ngram: int=3, 
    top_n=10, 
    diversity=0.9,
    driver: GraphDatabase.driver=driver,
    verbose: bool=True):
    """process a ticker and add that to the database"""
    metadata.update({'node_id': ticker})
    add_node(ticker, metadata=metadata, node_labels=['Company'], driver=driver,
        verbose=verbose)
    keywords = []
    for n in range(1, max_ngram+1):
        keywords += extract_keywords(metadata.get("description"), 
                ngram_range=(1, n), 
                top_n=top_n, diversity=diversity)

    for keyword, confidence in keywords:
        add_node(keyword, {'name': keyword, 
            'data_source': 'keyBERT extraction',
            'node_id': keyword}, 
            node_labels=['Topic'], driver=driver, verbose=verbose)
        # input()
        add_relation(source={'label': 'Company', 'node_id': ticker}, 
            target={'label': 'Topic', 'node_id': keyword}, 
            relation_props={'confidence': confidence,
                'weight': confidence},
            relation_type='mention',
            directed=True,
            verbose=verbose
                )
        # input()
    return keywords

In [91]:
node_exists(node_labels=['Topic'], prop={'node_id': 'resources'})

False

In [102]:
import logging
LOGFILE = "log/log.txt"
logging.basicConfig(filename=LOGFILE, level=logging.ERROR)
df = df.fillna('')
keywords = []
for _, row in df.iterrows():
    try:
        keywords += add_ticker_with_keyword(row.symbol, row.to_dict(), verbose=False)
    except Exception as e:
        logging.error(e)

In [85]:
keywords

[('shenzhen', 0.5195),
 ('resources', 0.2999),
 ('buildings', 0.2902),
 ('building', 0.2868),
 ('property', 0.2795),
 ('development', 0.2652),
 ('subsidiary', 0.2651),
 ('properties', 0.237),
 ('estate', 0.2314),
 ('construction', 0.215),
 ('shenzhen properties', 0.6845),
 ('shenzhen investment', 0.5943),
 ('of shenzhen', 0.5411),
 ('shenzhen', 0.5195),
 ('china shenzhen', 0.5),
 ('shenzhen china', 0.4941),
 ('in shenzhen', 0.4794),
 ('resources development', 0.442),
 ('properties resources', 0.416),
 ('property management', 0.4146),
 ('shenzhen properties resources', 0.7357),
 ('shenzhen properties', 0.6845),
 ('china shenzhen properties', 0.6823),
 ('shenzhen investment co', 0.6244),
 ('subsidiary of shenzhen', 0.6162),
 ('of shenzhen investment', 0.6121),
 ('based in shenzhen', 0.5967),
 ('shenzhen investment', 0.5943),
 ('of shenzhen', 0.5411),
 ('shenzhen china shenzhen', 0.5254)]